In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# --- 1. Load data ---
file_path = "/data/crawl_tiki_data/books_data_2025-10-01_14-30-45.csv"
df = pd.read_csv(file_path)

df['content'] = (
    df['name'].fillna('') + ' ' +
    df['authors'].fillna('') + ' ' +
    df['category'].fillna('') + ' ' +
    df['short_description'].fillna('')
)

In [5]:
# --- 2. Load model ---
model = SentenceTransformer('keepitreal/vietnamese-sbert')

D:\LightRAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--keepitreal--vietnamese-sbert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For

In [6]:
# --- 3. Encode embeddings ---
print("🔄 Đang sinh embedding...")
embeddings = model.encode(df['content'], convert_to_tensor=False, show_progress_bar=True)
title_embeddings = model.encode(df['name'].fillna('').tolist(), convert_to_tensor=False, show_progress_bar=True)
authors_embeddings = model.encode(df['authors'].fillna('').tolist(), convert_to_tensor=False, show_progress_bar=True)
category_embeddings = model.encode(df['category'].fillna('').tolist(), convert_to_tensor=False, show_progress_bar=True)

cosine_sim = cosine_similarity(embeddings)

🔄 Đang sinh embedding...


Batches: 100%|██████████| 63/63 [00:10<00:00,  6.14it/s]


In [7]:
# --- 4. Phát hiện loại query ---
def detect_query_type(query):
    query_emb = model.encode(query, convert_to_tensor=False)

    sim_title = cosine_similarity([query_emb], title_embeddings)[0].max()
    sim_author = cosine_similarity([query_emb], authors_embeddings)[0].max()
    sim_category = cosine_similarity([query_emb], category_embeddings)[0].max()
    sim_content = cosine_similarity([query_emb], embeddings)[0].max()

    sims = {
        "title": sim_title,
        "author": sim_author,
        "category": sim_category,
        "content": sim_content
    }
    query_type = max(sims, key=sims.get)
    return query_type, sims[query_type]


In [8]:
# --- 5. Gộp các loại embedding ---
def find_best_match_across_embeddings(query_text):
    query_embedding = model.encode(query_text, convert_to_tensor=False)
    sim_title = cosine_similarity([query_embedding], title_embeddings)[0]
    sim_content = cosine_similarity([query_embedding], embeddings)[0]
    sim_author = cosine_similarity([query_embedding], authors_embeddings)[0]
    sim_category = cosine_similarity([query_embedding], category_embeddings)[0]

    combined_sims = (
        0.25 * sim_title +
        0.25 * sim_content +
        0.25 * sim_author +
        0.25 * sim_category
    )
    top_idx = np.argmax(combined_sims)
    top_score = combined_sims[top_idx]
    return top_idx, top_score

In [9]:
# --- 6. Gợi ý sách ---
def recommend_books(query_text, top_n=5, threshold=0.5):
    query_text = query_text.strip()
    
    #  Xác định loại query (author / category / title / content)
    query_type, confidence = detect_query_type(query_text)
    print(f" Nhận diện query thuộc loại: {query_type} (độ tin cậy: {confidence:.2f})")

    #  Trường hợp query là tác giả
    if query_type == "author":
        idx = np.argmax(cosine_similarity([model.encode(query_text)], authors_embeddings)[0])
        author_name = df.loc[idx, 'authors']
        print(f" Gợi ý các sách của tác giả: {author_name}")
        result = df[df['authors'].str.contains(author_name, case=False, na=False)]

    #  Trường hợp query là thể loại
    elif query_type == "category":
        idx = np.argmax(cosine_similarity([model.encode(query_text)], category_embeddings)[0])
        category_name = df.loc[idx, 'category']
        print(f" Gợi ý các sách thuộc thể loại: {category_name}")
        result = df[df['category'].str.contains(category_name, case=False, na=False)]

    #  Còn lại (title hoặc content)
    else:
        #  Kiểm tra khớp trực tiếp tên sách (ưu tiên nhất)
        matches = df[df['name'].str.contains(query_text, case=False, na=False)]
        if not matches.empty:
            print(f" Khớp chính xác theo tên sách: {matches.iloc[0]['name']}")
            result = matches[['name', 'authors', 'category', 'link']].head(top_n).copy()
            return result
            
        print(" Truy vấn được xem là liên quan đến tiêu đề hoặc nội dung.")
        query_emb = model.encode(query_text, convert_to_tensor=False)

        # Tính tương đồng dựa trên content tổng hợp (tên + mô tả)
        sim_content = cosine_similarity([query_emb], embeddings)[0]

        top_idx = np.argmax(sim_content)
        top_score = sim_content[top_idx]

        if top_score < threshold:
            print(" Không có match tốt — gợi ý ngẫu nhiên.")
            result = df.sample(top_n)
        else:
            print(f" Tìm thấy sách gần nhất: {df.loc[top_idx, 'name']} (similarity: {top_score:.2f})")

            # Lấy top N sách tương tự
            sim_scores = list(enumerate(sim_content))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n + 1]
            result = df.iloc[[i[0] for i in sim_scores]]

    # Chuẩn hóa kết quả
    result = result[['name', 'authors', 'category', 'link']].head(top_n).copy()
    return result


In [11]:
# --- 7. Test ---
if __name__ == "__main__":
    query = input("Nhập nội dung/tên/tác giả/thể loại: ")
    recs = recommend_books(query, top_n=5)
    print("\n Top sách gợi ý:\n")
    print(recs.to_string(index=False))

 Nhận diện query thuộc loại: title (độ tin cậy: 0.51)
 Truy vấn được xem là liên quan đến tiêu đề hoặc nội dung.
 Không có match tốt — gợi ý ngẫu nhiên.

 Top sách gợi ý:

                                          name                authors                      category                                                   link
                           Thép Đã Tôi Thế Đấy   Nikolai A. Ostrovsky Tác Phẩm Kinh Điển Nước Ngoài https://tiki.vn/product-p195545297.html?spid=195545299
                              Gió Lạnh Đầu Mùa              Thạch Lam            Bút Ký - Tự Truyện https://tiki.vn/product-p195545261.html?spid=195545262
                 Brian Tracy - Thuật Hùng Biện            Brian Tracy         Sách kỹ năng làm việc https://tiki.vn/product-p208198122.html?spid=208198124
                            Trò Chơi Giẫm Bóng           Okamoto Kido                Truyện kinh dị https://tiki.vn/product-p273962508.html?spid=273962511
7 Câu Hỏi "Thần Kỳ" Của Mọi Sếp Giỏi (Tái bản) Michae